In [30]:
import pandas as pd
import numpy as np

In [2]:
df_node_label = pd.read_csv('tmp/dgl/AmazonCoBuyComputerDataset_node_labels.csv')
print(df_node_label.shape)
df_node_label.head()

(13752, 2)


,node_id,label
0,0,8
1,1,8
2,2,1
3,3,8
4,4,4


In [61]:
np.random.seed(seed=1234)
is_test = np.random.binomial(n=1, p=0.2, size=df_node_label.shape[0])
sum(is_test)

2731

In [65]:
sum(is_test) / df_node_label.shape[0]

0.1985892961023851

In [62]:
df_node_label['is_test'] = is_test

In [63]:
df_node_label.head()

,node_id,label,is_test
0,0,8,0
1,1,8,0
2,2,1,0
3,3,8,0
4,4,4,0


In [3]:
df_node_features = pd.read_csv('tmp/dgl/AmazonCoBuyComputerDataset_node_features.csv')
print(df_node_features.shape)
df_node_features.head()

(13752, 768)


,node_id,0,1,2,3,4,5,6,7,8,...,757,758,759,760,761,762,763,764,765,766
0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0


In [6]:
df_edge = pd.read_csv('tmp/dgl/AmazonCoBuyComputerDataset_edge_list.csv')
print(df_edge.shape)
df_edge = df_edge.rename(columns={'src':'src_node_id', 'dst':'dst_node_id'})
df_edge.head()

(491722, 2)


,src_node_id,dst_node_id
0,1,0
1,2,0
2,3,0
3,0,1
4,2,1


In [7]:
df_edge['src_node_id'].nunique(), df_edge['dst_node_id'].nunique()

(13471, 13471)

13752 - 13471 = 281の商品が接続を持たない

In [12]:
df_edge[df_edge['src_node_id']==0]

,src_node_id,dst_node_id
3,0,1
6,0,2
9,0,3


In [14]:
df_edge[df_edge['src_node_id']==1]

,src_node_id,dst_node_id
0,1,0
7,1,2
10,1,3


双方向で矢印が貼られている  
片方に削除

In [20]:
df_edge.shape[0] / 2

245861.0

In [15]:
pairs = set(tuple(sorted(pair)) for pair in zip(df_edge['src_node_id'], df_edge['dst_node_id']))
len(pairs)

245861

In [24]:
#pairs

In [21]:
src, dst = [], []
for pair in pairs:
    src.append(pair[0])
    dst.append(pair[1])
len(src), len(dst)

(245861, 245861)

In [23]:
df_edge2 = pd.DataFrame({'src_node_id':src, 'dst_node_id':dst})
print(df_edge2.shape)
df_edge2.head()

(245861, 2)


,src_node_id,dst_node_id
0,11203,11714
1,11705,12864
2,11010,11319
3,11450,11591
4,8690,9773


In [25]:
df_edge2['src_node_id'].nunique(), df_edge2['dst_node_id'].nunique()

(13438, 11267)

In [26]:
df_edge2.describe()

,src_node_id,dst_node_id
count,245861.000000,245861.000000
mean,7313.910889,9885.137944
std,3285.517776,3198.647637
min,0.000000,1.000000
25%,5209.000000,7732.000000
50%,7080.000000,10997.000000
75%,9890.000000,12661.000000
max,13469.000000,13470.000000


In [27]:
df_edge2[df_edge2['src_node_id']==0]

,src_node_id,dst_node_id
3346,0,3
95030,0,1
236669,0,2


In [28]:
df_edge2[df_edge2['src_node_id']==1]

,src_node_id,dst_node_id
746,1,3
234129,1,2


# Hydrogen Torch

必要データ
- train_df.csv
- test_df.csv
- meta_node.csv
- goods.csv
- meta_relation.csv
- bought_together.csv

In [66]:
# df_node_label
# df_node_features
# df_edge2

In [70]:
train_df = df_node_label[df_node_label['is_test'] == 0][['node_id','label']]
test_df = df_node_label[df_node_label['is_test'] == 1][['node_id','label']]
train_df.shape, test_df.shape

((11021, 2), (2731, 2))

In [71]:
train_df['label'].value_counts()

4    4169
8    1754
1    1694
2    1111
7     663
3     430
6     376
0     348
5     242
9     234
Name: label, dtype: int64

In [72]:
test_df['label'].value_counts()

4    989
1    448
8    402
2    303
7    155
3    112
6    111
0     88
5     66
9     57
Name: label, dtype: int64

In [77]:
meta_node = pd.DataFrame({'node_type':['goods'], 'file_name':['goods.csv'], 'is_target':[1]})
meta_node

,node_type,file_name,is_target
0,goods,goods.csv,1


In [79]:
goods = df_node_features
goods.head()

,node_id,0,1,2,3,4,5,6,7,8,...,757,758,759,760,761,762,763,764,765,766
0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0


In [87]:
meta_relation = pd.DataFrame({'src_node_type':['goods'], 
                              'dst_node_type':['goods'], 
                              'edge_type':['bought_together'],
                              'file_name':['bought_together.csv'],
                              'is_directional':[0]})
meta_relation

,src_node_type,dst_node_type,edge_type,file_name,is_directional
0,goods,goods,bought_together,bought_together.csv,0


In [88]:
bought_together = df_edge2
bought_together.head()

,src_node_id,dst_node_id
0,11203,11714
1,11705,12864
2,11010,11319
3,11450,11591
4,8690,9773


In [92]:
!ls tmp

HT_AmazonCoBuyComputerDataset dgl


In [93]:
train_df.to_csv('tmp/HT_AmazonCoBuyComputerDataset/train_df.csv', index=False)
test_df.to_csv('tmp/HT_AmazonCoBuyComputerDataset/test_df.csv', index=False)
meta_node.to_csv('tmp/HT_AmazonCoBuyComputerDataset/meta_node.csv', index=False)
goods.to_csv('tmp/HT_AmazonCoBuyComputerDataset/goods.csv', index=False)
meta_relation.to_csv('tmp/HT_AmazonCoBuyComputerDataset/meta_relation.csv', index=False)
bought_together.to_csv('tmp/HT_AmazonCoBuyComputerDataset/bought_together.csv', index=False)

# Driverless AI

In [ ]:
# df_node_label
# df_node_features

In [96]:
df_node_label.shape, df_node_features.shape

((13752, 3), (13752, 768))

In [102]:
df_node_label.head()

,node_id,label,is_test
0,0,8,0
1,1,8,0
2,2,1,0
3,3,8,0
4,4,4,0


In [103]:
df_node_features.head()

,node_id,0,1,2,3,4,5,6,7,8,...,757,758,759,760,761,762,763,764,765,766
0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0


In [105]:
df_node = pd.concat([df_node_label,df_node_features.drop(columns='node_id')], axis=1)
print(df_node.shape)
df_node.head()

(13752, 770)


,node_id,label,is_test,0,1,2,3,4,5,6,...,757,758,759,760,761,762,763,764,765,766
0,0,8,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,8,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,3,8,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0


In [107]:
train_node_df = df_node[df_node['is_test'] == 0].drop(columns='is_test')
test_node_df = df_node[df_node['is_test'] == 1].drop(columns='is_test')
train_node_df.shape, test_node_df.shape

((11021, 769), (2731, 769))

In [108]:
train_node_df.head()

,node_id,label,0,1,2,3,4,5,6,7,...,757,758,759,760,761,762,763,764,765,766
0,0,8,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,3,8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0


In [109]:
test_node_df.head()

,node_id,label,0,1,2,3,4,5,6,7,...,757,758,759,760,761,762,763,764,765,766
7,7,8,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
8,8,8,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
9,9,8,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
19,19,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,24,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
!ls tmp

DAI_AmazonCoBuyComputerDataset    HT_AmazonCoBuyComputerDataset.zip
HT_AmazonCoBuyComputerDataset     dgl


In [111]:
train_node_df.to_csv('tmp/DAI_AmazonCoBuyComputerDataset/AmazonCoBuyComputerDataset_train.csv', index=False)
test_node_df.to_csv('tmp/DAI_AmazonCoBuyComputerDataset/AmazonCoBuyComputerDataset_test.csv', index=False)